In [4]:
import psycopg2
from faker import Faker
import random

fake = Faker()

def generate_password():
    return fake.password(length=12, special_chars=True, digits=True, upper_case=True, lower_case=True)

def create_table_if_not_exists(conn):
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS user_for_comparison (
        user_id SERIAL PRIMARY KEY, 
        firstname VARCHAR(255), 
        lastname VARCHAR(255), 
        email VARCHAR(255) UNIQUE, 
        username VARCHAR(255) UNIQUE CHECK (length(username) > 8), 
        password VARCHAR(255) CHECK (length(password) > 8), 
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    conn.commit()
    cur.close()

def insert_fake_data(conn, n=100000):
    cur = conn.cursor()
    for _ in range(n):
        firstname = fake.first_name()
        lastname = fake.last_name()
        email = fake.unique.email()
        while True:
            username = fake.unique.user_name()
            if len(username) <= 8:
                username += str(random.randint(1000, 9999))
            else:
                break
        password = generate_password() 
        cur.execute(
            "INSERT INTO user_for_comparison (firstname, lastname, email, username, password) VALUES (%s, %s, %s, %s, %s)",
            (firstname, lastname, email, username, password)
        )
    conn.commit()
    cur.close()


conn = psycopg2.connect(
    dbname="app_auth",
    user="user",
    password="root",
    host="localhost",
    port="25432"
)

create_table_if_not_exists(conn)

insert_fake_data(conn)

conn.close()


In [5]:
import psycopg2
import time

conn = psycopg2.connect(
    dbname="app_auth",
    user="user",
    password="root",
    host="localhost",
    port="25432"
)

def measure_query_time(username):
    cur = conn.cursor()
    start_time = time.time()
    cur.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    result = cur.fetchall()
    end_time = time.time()
    duration = end_time - start_time
    print(f"Temps d'exécution pour l'utilisateur '{username}': {duration} secondes.")
    cur.close()

usernames_to_test = ["username1", "username2", "username3"]

for username in usernames_to_test:
    measure_query_time(username)

conn.close()


Temps d'exécution pour l'utilisateur 'username1': 0.019189119338989258 secondes.
Temps d'exécution pour l'utilisateur 'username2': 0.004366636276245117 secondes.
Temps d'exécution pour l'utilisateur 'username3': 0.005029439926147461 secondes.


In [6]:
import psycopg2
import time

def measure_query_time(conn, username):
    with conn.cursor() as cur:
        start_time = time.time()
        cur.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
        cur.fetchall()
        end_time = time.time()
        print(f"Temps d'exécution pour '{username}': {end_time - start_time:.4f} secondes.")

conn = psycopg2.connect(
    dbname="app_auth",
    user="user",
    password="root",
    host="localhost",
    port="25432"
)

with conn.cursor() as cur:
    cur.execute("CREATE INDEX IF NOT EXISTS idx_username ON user_for_comparison(username);")
    conn.commit()
print("Index créé avec succès sur la colonne username.")

usernames_to_test = ["username1", "username2", "username3"]

print("\nMesure du temps d'exécution après la création de l'index:")
for username in usernames_to_test:
    measure_query_time(conn, username)

conn.close()


Index créé avec succès sur la colonne username.

Mesure du temps d'exécution après la création de l'index:
Temps d'exécution pour 'username1': 0.0048 secondes.
Temps d'exécution pour 'username2': 0.0021 secondes.
Temps d'exécution pour 'username3': 0.0012 secondes.


## Scalingp